In [1]:
import numpy as np
import os
import tensorflow as tf
import joblib
import matplotlib.pyplot as plt

# --- Configurações Iniciais ---
TICKER_PARA_AVALIAR = "PETR4.SA"
CAMINHO_DADOS_PROCESSADOS = "../data/03_processed"
CAMINHO_MODELOS = "../models"

print("Ambiente configurado.")

2025-10-31 00:50:57.898806: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2025-10-31 00:50:57.971020: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/home/marolli/Projetos/TCC_2025/.venv/lib64/python3.13/site-packages/google/protobuf/runtime_version.py:98: UserWarning: Protobuf gencode version 5.28.3 is exactly one major version older than the runtime version 6.31.1 at tensorflow/core/framework/attr_value.proto. Please update the gencode to avoid compatibility violations in the next runtime release.
  warnings.warn(
/home/marolli/Projetos/TCC_2025/.venv/lib64/python3.13/site-packages/google/protobuf/runtime_version.py:98: UserWarning: Protobuf gencode version 5.28.3 is exactly one

Ambiente configurado.


In [2]:
# Carrega o modelo treinado
caminho_modelo = os.path.join(CAMINHO_MODELOS, f"{TICKER_PARA_AVALIAR}_lstm_model.h5")
model = tf.keras.models.load_model(caminho_modelo)

# Carrega o scaler
caminho_scaler = os.path.join(CAMINHO_MODELOS, f"{TICKER_PARA_AVALIAR}_scaler.pkl")
scaler = joblib.load(caminho_scaler)

# Carrega os dados de teste (X_test para prever, y_test para comparar)
X_test = np.load(os.path.join(CAMINHO_DADOS_PROCESSADOS, f'{TICKER_PARA_AVALIAR}_X_test.npy'))
y_test = np.load(os.path.join(CAMINHO_DADOS_PROCESSADOS, f'{TICKER_PARA_AVALIAR}_y_test.npy'))

print("Modelo, scaler e dados de teste carregados com sucesso.")
print("Formato de X_test:", X_test.shape)

2025-10-31 00:51:01.760307: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


Modelo, scaler e dados de teste carregados com sucesso.
Formato de X_test: (1238, 60, 1)


In [3]:
# Pede ao modelo para fazer previsões com os dados de teste
previsoes_normalizadas = model.predict(X_test)

# Agora, o passo mais importante: reverter a normalização para a escala de preço original
# Usamos o MESMO scaler que usamos para normalizar os dados
previsoes_reais = scaler.inverse_transform(previsoes_normalizadas)
y_test_reais = scaler.inverse_transform(y_test.reshape(-1, 1))

print("Previsões realizadas e desnormalizadas.")

39/39 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step
Previsões realizadas e desnormalizadas.


In [4]:
# CÉLULA DE PREPARAÇÃO PARA O GRÁFICO - ADICIONE ANTES DO CÓDIGO DO PLOTLY

import pandas as pd # Garante que o pandas está importado nesta célula

print("Preparando dados para o eixo X do gráfico...")

# 1. Recarregar o arquivo CSV original para termos acesso ao DataFrame 'df' com as datas
caminho_arquivo_original = os.path.join("../data/01_raw", f"{TICKER_PARA_AVALIAR}.csv")
df = pd.read_csv(
    caminho_arquivo_original,
    header=None,
    skiprows=3,
    index_col=0,
    parse_dates=[0]
)
df.dropna(inplace=True)

# 2. Recalcular o ponto_divisao, exatamente como fizemos antes
dados_fechamento_temp = df.iloc[:, 1].values.reshape(-1, 1) # Usando a coluna de posição 1 (Close)
PERCENTUAL_TREINO = 0.8
ponto_divisao = int(len(dados_fechamento_temp) * PERCENTUAL_TREINO)

# 3. Garantir que a JANELA_DE_TEMPO está definida
JANELA_DE_TEMPO = 60

print("Variáveis 'df', 'ponto_divisao' e 'JANELA_DE_TEMPO' prontas para o gráfico!")

Preparando dados para o eixo X do gráfico...
Variáveis 'df', 'ponto_divisao' e 'JANELA_DE_TEMPO' prontas para o gráfico!


In [6]:
# Célula para o Gráfico Interativo (substitui a célula do matplotlib)

import plotly.graph_objects as go

# --- Preparação dos Dados para o Gráfico ---

# O 'y_test_reais' e 'previsoes_reais' já temos da célula anterior.
# Para o eixo X, vamos pegar as datas reais correspondentes ao período de teste.
# Lembre-se que o 'df' é o nosso DataFrame original carregado no notebook de padronização.
# Precisamos garantir que ele está carregado ou recarregá-lo.
# Se você não tem mais a variável 'df', descomente e execute as 3 linhas abaixo.
# caminho_arquivo = os.path.join("../data/01_raw", f"{TICKER_PARA_AVALIAR}.csv")
# df = pd.read_csv(caminho_arquivo, header=None, skiprows=3, index_col=0, parse_dates=[0])
# df.dropna(inplace=True)

# Pega as datas do índice do DataFrame que correspondem ao nosso conjunto de teste
datas_teste = df.index[ponto_divisao:]

# Como nossas previsões (X_test) começam após a primeira janela de 60 dias,
# precisamos ajustar as datas para alinharem corretamente com as previsões.
datas_para_plotar = datas_teste[JANELA_DE_TEMPO:]


# --- Criação do Gráfico Interativo ---

# Cria uma figura em branco
fig = go.Figure()

# Adiciona a linha de "Preço Real" à figura
fig.add_trace(go.Scatter(
    x=datas_para_plotar,
    y=y_test_reais.flatten(), # .flatten() transforma o array em uma lista simples
    mode='lines',
    name='Preço Real',
    line=dict(color='blue')
))

# Adiciona a linha de "Previsão do Modelo" à figura
fig.add_trace(go.Scatter(
    x=datas_para_plotar,
    y=previsoes_reais.flatten(),
    mode='lines',
    name='Previsão do Modelo',
    line=dict(color='red', dash='dash')
))

# Atualiza o layout do gráfico com títulos, eixos e um tema escuro
fig.update_layout(
    title=f'Comparação Interativa: Preço Real vs. Previsão para {TICKER_PARA_AVALIAR}',
    xaxis_title='Data',
    yaxis_title='Preço de Fechamento (R$)',
    legend_title='Legenda',
    template='plotly_dark' # Um tema visualmente agradável
)

# Mostra a figura interativa
fig.show()